# Introduction

The ever-expanding world of movies offers rich and interconnected data, making it an ideal candidate for building a knowledge graph. In this project, we leverage Neo4j, a powerful graph database, and GROQ API to construct an intelligent knowledge graph using a dataset comprising movie details such as movie IDs, release years, titles, actors, directors, genres, and IMDb ratings. By integrating these technologies, we aim to enable advanced querying, semantic understanding, and insights into movie data. This project highlights the potential of graph databases and APIs in managing complex and highly relational datasets.

# LIBRARY REQUIRED

In [1]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.9 MB/s eta 0:00:00


In [9]:
!pip install --upgrade --quiet langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 5.0 MB/s eta 0:00:00


# Graphdb configuration

## Steps to Create a Free Neo4j Instance
1. Visit Neo4j AuraDB:
  * Go to https://neo4j.com/cloud/aura/.

2. Log In or Sign Up:
  * If you already have an account, log in.
  * If not, click "Sign Up" to create a free account.

3. Create a New Database:
  * Click the "Create a Database" button on the dashboard.

4. Select the Free Tier:
  * Choose the "Aura Free" option for the free-tier database.

5. Configure Your Database:
  * Database Name: Enter a unique name for your database.
  * Cloud Region: Select the nearest cloud region for optimal performance.

6. Launch the Database:
  * Click "Create".
  * Wait for the database to initialize (this may take a minute or two).

7. Access Connection Details:
  * Once the instance is ready, click on it to view its connection URI, username, and password.

In [2]:
## Graphdb configuration
NEO4J_URI = " " # put your connection URI
NEO4J_USERNAME = " " # put your username
NEO4J_PASSWORD = " " # put your password


In [3]:
import os
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [4]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [5]:
graph

# Extract Information

## Steps to Obtain a GROQ API Key
1. Go to GROQ's API Platform:
  * Visit the official GROQ API website or portal where API keys are managed.

2. Sign Up or Log In:
  * Create an account if you don’t have one.
  * Log in if you already have an account.

3. Navigate to API Keys:
  * In the dashboard, find the section labeled API Keys or Developer Tools.

4. Generate a Key:
  * Click "Create New API Key" or "Generate Key".
  * Optionally, provide a name or description for the key.

5. Save the Key:
  * Copy the generated API key immediately (it’s often only shown once).
  * Store it securely in a password manager or environment variable.

In [6]:
groq_api_key=" " # put your own API key

In [7]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7c59e13c3220>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7c59e13c14e0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_core.documents import Document
text="""
Elon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space
company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,
formerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.
He is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be
US$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended
the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through
his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.
Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics
 and physics. He moved to California in 1995 to attend Stanford University, but dropped out after
  two days and, with his brother Kimbal, co-founded online city guide software company Zip2.
 """
documents=[Document(page_content=text)]
documents

[Document(metadata={}, page_content="\nElon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space\ncompany SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,\nformerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.\nHe is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be\nUS$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended\nthe University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through\nhis Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.\nMusk later transferred to the University of Pennsylvania and received bachelor's degrees in economics\n and physics. He moved to California in 1995 to attend Stanford University, but dropped out after\n  two days and, with his brother Kimbal, co-founded onlin

In [10]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

In [11]:
graph_documents=llm_transformer.convert_to_graph_documents(documents)

In [12]:
graph_documents

[GraphDocument(nodes=[Node(id='Elon Reeve Musk', type='Person', properties={}), Node(id='Maye', type='Person', properties={}), Node(id='Errol Musk', type='Person', properties={}), Node(id='Kimbal', type='Person', properties={}), Node(id='Spacex', type='Organization', properties={}), Node(id='Tesla, Inc.', type='Organization', properties={}), Node(id='X Corp.', type='Organization', properties={}), Node(id='The Boring Company', type='Organization', properties={}), Node(id='Xai', type='Organization', properties={}), Node(id='Neuralink', type='Organization', properties={}), Node(id='Openai', type='Organization', properties={}), Node(id='University Of Pretoria', type='Educational_institution', properties={}), Node(id="Queen'S University", type='Educational_institution', properties={}), Node(id='University Of Pennsylvania', type='Educational_institution', properties={}), Node(id='Stanford University', type='Educational_institution', properties={})], relationships=[Relationship(source=Node(id

In [13]:
graph_documents[0].nodes

[Node(id='Elon Reeve Musk', type='Person', properties={}),
 Node(id='Maye', type='Person', properties={}),
 Node(id='Errol Musk', type='Person', properties={}),
 Node(id='Kimbal', type='Person', properties={}),
 Node(id='Spacex', type='Organization', properties={}),
 Node(id='Tesla, Inc.', type='Organization', properties={}),
 Node(id='X Corp.', type='Organization', properties={}),
 Node(id='The Boring Company', type='Organization', properties={}),
 Node(id='Xai', type='Organization', properties={}),
 Node(id='Neuralink', type='Organization', properties={}),
 Node(id='Openai', type='Organization', properties={}),
 Node(id='University Of Pretoria', type='Educational_institution', properties={}),
 Node(id="Queen'S University", type='Educational_institution', properties={}),
 Node(id='University Of Pennsylvania', type='Educational_institution', properties={}),
 Node(id='Stanford University', type='Educational_institution', properties={})]

In [14]:
graph_documents[0].relationships

[Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Maye', type='Person', properties={}), type='PARENT', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Errol Musk', type='Person', properties={}), type='PARENT', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Kimbal', type='Person', properties={}), type='SIBLING', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Spacex', type='Organization', properties={}), type='INVOLVED_WITH', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Tesla, Inc.', type='Organization', properties={}), type='INVOLVED_WITH', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='X Corp.', type='Organization', properties

# Load the dataset of movie

In [15]:
### Load the dataset of movie

movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [16]:
graph

In [17]:
graph.query(movie_query)

[]

In [18]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


# Initializing the Graph Cypher Chain

In [21]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7c5a0bd0dd20>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7c59e13c3220>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7c59e13c14e0>, model_name='Gemma2-9b-I

## Executing the Query Using GraphCypherChain

In [22]:
response=chain.invoke({"query":"Who was the director of the moview GoldenEye"})

response




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})<-[:DIRECTED]-(p:Person) RETURN p.name  

Full Context:
[{'p.name': 'Martin Campbell'}]

> Finished chain.


{'query': 'Who was the director of the moview GoldenEye',
 'result': 'Martin Campbell  \n'}

In [23]:
response=chain.invoke({"query":"tell me the genre of th movie GoldenEye"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: 'GoldenEye'})<-[:IN_GENRE]-(g:Genre) RETURN g.name  

Full Context:
[]

> Finished chain.


{'query': 'tell me the genre of th movie GoldenEye',
 'result': "I don't know the answer. \n"}

In [24]:
response=chain.invoke({"query":"Who was the director in movie Casino"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: 'Casino'})<-[:DIRECTED]-(p:Person) RETURN p.name

Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


{'query': 'Who was the director in movie Casino',
 'result': 'Martin Scorsese  \n'}

In [25]:
response=chain.invoke({"query":"Which movie were released in 2008"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)
WHERE m.released = "2008"
RETURN m.title
Full Context:
[]

> Finished chain.


{'query': 'Which movie were released in 2008',
 'result': "I don't know the answer. \n"}

In [26]:
response=chain.invoke({"query":"Give me the list of movie having imdb rating more than 8"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie) WHERE m.imdbRating > 8 RETURN m.title

Full Context:
[{'m.title': 'Toy Story'}, {'m.title': 'Heat'}, {'m.title': 'Casino'}, {'m.title': 'Twelve Monkeys (a.k.a. 12 Monkeys)'}, {'m.title': 'Seven (a.k.a. Se7en)'}, {'m.title': 'Usual Suspects, The'}, {'m.title': 'Hate (Haine, La)'}, {'m.title': 'Braveheart'}, {'m.title': 'Taxi Driver'}, {'m.title': 'Anne Frank Remembered'}]

> Finished chain.


{'query': 'Give me the list of movie having imdb rating more than 8',
 'result': "I don't know the answer. \n"}

In [27]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]

# Conclusion

This project demonstrates the effectiveness of combining Neo4j and GROQ API in constructing a robust knowledge graph for movie data. The resulting graph allows for efficient storage, retrieval, and exploration of interconnected movie-related information. By utilizing graph queries and advanced API interactions, users can extract meaningful insights, such as identifying key contributors to cinema, genre-based trends, and relationships between actors and directors. This knowledge graph can serve as a foundation for further development in recommendation systems, data visualization, or enriched movie-related applications.